In [1]:
import pandas as pd
from lib.database.database_connector import DatabaseConnector

database_connector = DatabaseConnector("./data/database.db")
cursor = database_connector.cursor

In [2]:
data = pd.read_csv('data/auxiliary/wrds/financial_ratios_all.csv')
sp_500 = cursor.execute(
    "SELECT ticker FROM embeddings;"
).fetchall()
sp_500 = list(set(sp_500))
sp_500 = [x[0] for x in sp_500]

In [3]:
data.head()

,gvkey,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,...,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward,TICKER,cusip
0,1004,54594,2009-05-31,2009-11-30,2010-01-31,17.934,1.077,7.981,14.758,15.243,...,0.0,0.0,-0.043,1.152,2.250,NaN,-0.632,1.341,AIR,00036110
1,1004,54594,2009-05-31,2009-11-30,2010-02-28,17.554,1.077,7.981,14.446,14.921,...,0.0,0.0,-0.043,1.128,2.203,NaN,-0.584,1.340,AIR,00036110
2,1004,54594,2009-05-31,2009-11-30,2010-03-31,19.257,1.077,7.981,15.809,16.329,...,0.0,0.0,-0.043,1.237,2.410,NaN,-0.574,1.467,AIR,00036110
3,1004,54594,2009-05-31,2010-02-28,2010-04-30,18.304,0.898,9.293,16.146,16.473,...,0.0,0.0,-0.087,1.197,31.895,NaN,-0.539,1.548,AIR,00036110
4,1004,54594,2009-05-31,2010-02-28,2010-05-31,14.791,0.898,9.293,13.046,13.311,...,0.0,0.0,-0.087,0.968,25.772,NaN,-0.436,1.251,AIR,00036110


In [4]:
threshold = 0.1  # yüzde 30
nan_ratio = data.isna().mean()  # her sütun için NaN oranı
columns_to_drop = nan_ratio[nan_ratio > threshold].index  # %30'dan fazla olanlar
data = data.drop(columns=columns_to_drop)

In [5]:
data.head()

,gvkey,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,...,pay_turn,sale_invcap,sale_equity,rd_sale,adv_sale,staff_sale,accrual,ptb,TICKER,cusip
0,1004,54594,2009-05-31,2009-11-30,2010-01-31,17.934,1.077,7.981,14.758,15.243,...,11.295,1.298,1.992,0.0,0.0,0.0,-0.043,1.152,AIR,00036110
1,1004,54594,2009-05-31,2009-11-30,2010-02-28,17.554,1.077,7.981,14.446,14.921,...,11.295,1.298,1.992,0.0,0.0,0.0,-0.043,1.128,AIR,00036110
2,1004,54594,2009-05-31,2009-11-30,2010-03-31,19.257,1.077,7.981,15.809,16.329,...,11.295,1.298,1.992,0.0,0.0,0.0,-0.043,1.237,AIR,00036110
3,1004,54594,2009-05-31,2010-02-28,2010-04-30,18.304,0.898,9.293,16.146,16.473,...,10.717,1.265,1.872,0.0,0.0,0.0,-0.087,1.197,AIR,00036110
4,1004,54594,2009-05-31,2010-02-28,2010-05-31,14.791,0.898,9.293,13.046,13.311,...,10.717,1.265,1.872,0.0,0.0,0.0,-0.087,0.968,AIR,00036110


In [6]:
data['date'] = data['public_date'].apply(lambda x: x[:-3])

In [7]:
expected_date_list = []
for year in range(2010, 2025):
    for month in range(1, 13):
        expected_date = f"{year}-{month:02d}"
        expected_date_list.append(expected_date)

In [8]:
len(data)

665283

In [9]:
data = data[data['TICKER'].isin(sp_500)]

In [10]:
len(data)

78466

In [11]:
missing_rows = []

# Her TICKER için
for ticker in data['TICKER'].unique():
    # TICKER için mevcut tarihler
    existing_dates = set(data[data['TICKER'] == ticker]['date'])

    # Eksik tarihleri belirle
    missing_dates = set(expected_date_list) - existing_dates

    # Her eksik tarih için boş satır oluştur
    for missing_date in missing_dates:
        new_row = {
            'TICKER': ticker,
            'date': missing_date
        }
        missing_rows.append(new_row)

# Yeni boş satırları DataFrame'e çevir
missing_df = pd.DataFrame(missing_rows)

# Orijinal veriyle birleştir
data = pd.concat([data, missing_df], ignore_index=True)

# Tarih sıralaması yap (isteğe bağlı)
data = data.sort_values(by=['TICKER', 'date']).reset_index(drop=True)

In [12]:
len(data)

86400

In [13]:
data.isna().sum()

gvkey              7934
permno             7934
adate              7991
qdate              7934
public_date        7934
CAPEI              8203
bm                11037
evm                8328
pe_op_basic        8648
pe_op_dil          8655
pe_exi             8624
pe_inc             8621
ps                 7991
pcf                8062
npm                7991
opmbd              7991
opmad              7991
gpm                7991
ptpm               7991
cfm                8185
roa                8059
roe               11114
roce               8285
aftret_eq          8059
aftret_invcapx     8481
aftret_equity      8059
GProf              7990
equity_invcap      8007
debt_invcap        8168
totdebt_invcap     8197
capital_ratio      8145
cash_lt            8028
debt_at            8174
debt_ebitda        8518
lt_debt            8145
cash_debt          8430
lt_ppent           8839
dltt_be           11040
debt_assets        8028
debt_capital       8628
de_ratio           8028
at_turn         

In [14]:
data = data.groupby('TICKER').apply(lambda group: group.ffill().bfill()).reset_index(drop=True)

/tmp/ipykernel_6699/2790136424.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('TICKER').apply(lambda group: group.ffill().bfill()).reset_index(drop=True)


In [16]:
data.isna().sum()

gvkey                0
permno               0
adate                0
qdate                0
public_date          0
CAPEI                0
bm                 540
evm                  0
pe_op_basic       1260
pe_op_dil         1260
pe_exi            1260
pe_inc            1260
ps                   0
pcf                  0
npm                  0
opmbd                0
opmad                0
gpm                  0
ptpm                 0
cfm                  0
roa                  0
roe                360
roce                 0
aftret_eq            0
aftret_invcapx       0
aftret_equity        0
GProf                0
equity_invcap        0
debt_invcap          0
totdebt_invcap       0
capital_ratio        0
cash_lt              0
debt_at              0
debt_ebitda          0
lt_debt              0
cash_debt            0
lt_ppent           540
dltt_be            540
debt_assets          0
debt_capital       540
de_ratio             0
at_turn              0
rect_turn          180
pay_turn   

In [17]:
# Hala NaN içeren sütunları kontrol et
nan_cols_after_fill = data.columns[data.isna().any()].tolist()

# Bu sütunları tamamen drop et (çünkü ffill + bfill işe yaramadı)
data = data.drop(columns=nan_cols_after_fill)

In [18]:
data.isna().sum().sum()

0

In [20]:
data.drop('date', axis=1, inplace=True)

In [21]:
data.to_csv('data/auxiliary/wrds/financial_ratios_all_processed.csv')